In [17]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import nibabel as nib
from os import listdir
from os.path import isfile, join
from scipy import signal
import sys

# current path
cur_path = os.getcwd()
# file name and path
file = "BB_003_3T_cmrr_mbep2d_diff_0pt2_1_task_18s_off_12s_on_20200710180104_8"
path_ref = cur_path + "/" + file

file_name = cur_path + "/" + file + "/dfMRI_raw.nii.gz"
# loading nifti object/header
raw_data_ = nib.load(file_name)
header  = raw_data_.header
raw_data = raw_data_.get_fdata()

### 1. Motion correction
#### 1.1 Split b0 and b1 values

In [5]:
def gzip(filename: str, zip: bool) -> str:
    """
    Function that zips or unzips the filename file
    Args:
        filename (str)
        zip (bool): true -> zip, false -> unzip
    Returns:
        new_filename (str)
    """
    if zip == False:
        new_filename = filename.replace('.gz', '')
        os.system(f"gzip -f -c -d {filename} > {new_filename}")
    elif zip == True:
        os.system(f"gzip {filename} -f")
        new_filename = f'{filename}.gz'
    else:
        new_filename = None
        
    return new_filename


In [30]:
import nipype.interfaces.spm as spm
import zipfile

dfmri_degibbs_sdc = nib.load(f'{cur_path}/data/topup/dfmri_degibbs_sdc.nii.gz')
dfmri_degibbs_sdc = dfmri_degibbs_sdc.get_fdata()
for i in range(2):
    dfmri_degibbs_sdc_b = dfmri_degibbs_sdc[...,np.arange(i, dfmri_degibbs_sdc.shape[3], 2)]
    img_deggibs_sdc_b = nib.Nifti1Image(dfmri_degibbs_sdc_b, np.eye(4))
    nib.save(img_deggibs_sdc_b, f'{cur_path}/data/ADC/dfmri_degibbs_sdc_b{i}.nii.gz') 

    gzip(f'{cur_path}/data/ADC/dfmri_degibbs_sdc_b{i}.nii.gz', zip=False)
 

### 1.2 Apply spm for motion correction 
#### Problem : I don't have a matlab licence

In [27]:
# Matlab is needed
realign = spm.Realign()
print(type(f"{cur_path}/data/ADC/dfmri_degibbs_sdc_b0.nii.gz"))
realign.inputs.in_files = f"{cur_path}/data/ADC/dfmri_degibbs_sdc_b0.nii"
realign.run()

<class 'str'>


TraitError: Each element of the 'in_files' trait of a RealignInputSpec instance must be a pathlike object or string representing an existing file or a list of items which are a pathlike object or string representing an existing file, but a value of 'dfmri_degibbs_sdc_b0.nii.gz' <class 'str'> was specified.

In [16]:
# Test fsl mcFlirt instead
# Output_name : input_name_mcf
for i in range(2):
    os.system(f'mcflirt -in {cur_path}/data/ADC/dfmri_degibbs_sdc_b{i}')


Params: 0.0142074 : -0.273128 : -0.0220701
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.00881476 : -0.273199 : 0.0424645
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.00789085 : -0.24911 : 0.0422303
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.00789085 : -0.2764 : 0.0880636
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.00789085 : -0.276489 : 0.0882184
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.0066754 : -0.276011 : 0.0260057
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.00757803 : -0.27585 : 0.00871066
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.0088635 : -0.275885 : 0.00860135
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.0088635 : -0.276439 : -0.00871254
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.0088635 : -0.276622 : 0.034125
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.0088635 : -0.276309 : 0.0918986
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.0068635 : -0.276093 : 0.0202615
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.0117649 : -0.276824 : 0.068065
Tolerances: 0.004 : 0.16 : 0.16
Params: 0.00799723 : -0.276777 : 0

### 1.3 Take the output from Wictor "rdfMRI_denoised_unringed_sdc_b_{i}.nii.gz" to do the ANTS rigid registration

In [32]:
# Since I don't have matlab, I take Wictor's output here. 
# Problem : rdfMRI_denoised_unringed_sdc_b_1 was overwritten in his pipeline
for i in range(2):
    rdfMRI_sdc_b = nib.load(f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_{i}.nii.gz')
    affine_b = rdfMRI_sdc_b.affine
    header_b = rdfMRI_sdc_b.header
    rdfMRI_sdc_b = rdfMRI_sdc_b.get_fdata()

    rdfMRI_sdc_b_mean = np.mean(rdfMRI_sdc_b, axis=3)
    img_rdfMRI_sdc_b_mean = nib.Nifti1Image(rdfMRI_sdc_b_mean, affine_b, header_b) 
    nib.save(img_rdfMRI_sdc_b_mean, f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_{i}_mean.nii.gz') 

In [21]:
# # Check that the inputs are the same
# suffix = ['','_mean']
# for i in range(4):
#     print(i%2, suffix[i%2])
#     ref = nib.load(f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_{i%2}{suffix[i%2]}_ref.nii.gz')
#     ref = ref.get_fdata()

#     test = nib.load(f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_{i%2}{suffix[i%2]}.nii.gz')
#     test = test.get_fdata()
#     assert np.array_equal(test, ref, equal_nan=True), f"The rdfMRI_denoised_unringed_sdc {i%2} {suffix[i%2]} differs"
#     print(f"The rdfMRI_denoised_unringed_sdc {i%2} {suffix[i%2]} ok")

0 
The rdfMRI_denoised_unringed_sdc 0  ok
1 _mean


AssertionError: The rdfMRI_denoised_unringed_sdc 1 _mean differs

### 1.4 Ants rigid registration, fixed image = rdfMRI_denoised_unringed_sdc_b_0_mean

In [62]:
# output : name of the 00GenericAffine.mat, warped image
os.system('antsRegistration --dimensionality 3 --float 0 '
        f' --output [{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_mean_in_0, {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_mean_in_0.nii.gz] '
        ' --interpolation Linear '
        ' --winsorize-image-intensities [0.01,0.99] ' 
        ' --use-histogram-matching 0 '
        ' --transform Rigid[0.0001] ' 
        f' --metric MI[{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_0_mean.nii.gz, {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_mean.nii.gz,1,32,Regular,0.25] '
        ' --convergence [10000x10000x10000x10000,1e-6,10] '
        ' --shrink-factors 4x4x2x1 ' 
        ' --smoothing-sigmas 2x2x1x0vox')

0

### 1.5 Apply the transform to rdfMRI_denoised_unringed_sdc_b_1

In [40]:
# -r : reference image
# -i : input image
# -e : input image time : 3 -> timeseries
# -t : transform
# -v : verbose
# -o : output
os.system(f'antsApplyTransforms -r {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_0_mean.nii.gz'
          f' -i {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1.nii.gz -e 3 '
          f'-t {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_mean_in_00GenericAffine.mat '
          f'-v -o {cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_reg.nii.gz')

Using double precision for computations.
Input time-series image: /home/localadmin/Documents/preprocessing/data/ADC/rdfMRI_denoised_unringed_sdc_b_1.nii.gz
Reference image: /home/localadmin/Documents/preprocessing/data/ADC/rdfMRI_denoised_unringed_sdc_b_0_mean.nii.gz
The composite transform comprises the following transforms (in order): 
  1. /home/localadmin/Documents/preprocessing/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_mean_in_00GenericAffine.mat (type = AffineTransform)
Default pixel value: 0
Interpolation type: LinearInterpolateImageFunction
  Applying transform(s) to timePoint/channel/dimension5 0 (out of 300).
  Applying transform(s) to timePoint/channel/dimension5 1 (out of 300).
  Applying transform(s) to timePoint/channel/dimension5 2 (out of 300).
  Applying transform(s) to timePoint/channel/dimension5 3 (out of 300).
  Applying transform(s) to timePoint/channel/dimension5 4 (out of 300).
  Applying transform(s) to timePoint/channel/dimension5 5 (out of 300).
  Applying tr

0

In [19]:
rdfMRI_sdc_b = nib.load(f'{path_ref}/rdfMRI_denoised_unringed_sdc_b_1.nii.gz')
affine_b = rdfMRI_sdc_b.affine
header_b = rdfMRI_sdc_b.header
rdfMRI_sdc_b = rdfMRI_sdc_b.get_fdata()

rdfMRI_sdc_b_mean = np.mean(rdfMRI_sdc_b, axis=3)
img_rdfMRI_sdc_b_mean = nib.Nifti1Image(rdfMRI_sdc_b_mean, affine_b, header_b) 
nib.save(img_rdfMRI_sdc_b_mean, f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_1_ref_mean.nii.gz') 

### 1.6 Put together b0 and b1 again

In [46]:
rdfMIR_denoised_unringed_sdc = np.zeros(header['dim'][1:5])
add = ['', '_reg']
for i in range(2):
    rdfMIR_denoised_unringed_sdc_b = nib.load(f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_{i}{add[i]}.nii.gz')
    rdfMIR_denoised_unringed_sdc_b = rdfMIR_denoised_unringed_sdc_b.get_fdata()
    rdfMIR_denoised_unringed_sdc[..., np.arange(i, rdfMIR_denoised_unringed_sdc.shape[3], 2)] = rdfMIR_denoised_unringed_sdc_b

del rdfMIR_denoised_unringed_sdc_b
img_rdfMIR_denoised_unringed_sdc = nib.Nifti1Image(rdfMIR_denoised_unringed_sdc, affine_b, header) 
nib.save(img_rdfMIR_denoised_unringed_sdc, f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc.nii.gz') 

In [20]:
img_test = nib.load(f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc_b_0.nii.gz')
img_test = img_test.get_fdata()
img_ref = nib.load(f'{path_ref}/rdfMRI_denoised_unringed_sdc_b_0.nii.gz')
img_ref = img_ref.get_fdata()

assert np.array_equal(img_test, img_ref, equal_nan=True), "The rdfMRI_denoised_unringed_sdc differs"

In [47]:
# Replace NaN values by 0
rdfMIR_denoised_unringed_sdc = np.nan_to_num(rdfMIR_denoised_unringed_sdc)

assert np.argwhere(np.isnan(rdfMIR_denoised_unringed_sdc)).size == 0, "There are still NaN values in the array"

img_rdfMIR_denoised_unringed_sdc = nib.Nifti1Image(rdfMIR_denoised_unringed_sdc, affine_b, header) 
nib.save(img_rdfMIR_denoised_unringed_sdc, f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc.nii.gz') 

del img_rdfMIR_denoised_unringed_sdc, rdfMIR_denoised_unringed_sdc


### 2. Making ADC maps

In [48]:
if 'b0_1' in file : b_val_diff = 1
elif '0pt2_1' in file : b_val_diff = 0.8
elif '0pt29_1pt45' in file: b_val_diff = 1.155

TR = header["pixdim"][4]

ardfMRI_denoised_unringed_sdc = nib.load(f'{cur_path}/data/ADC/rdfMRI_denoised_unringed_sdc.nii.gz')
ardfMRI_denoised_unringed_sdc = ardfMRI_denoised_unringed_sdc.get_fdata()
ardfMRI_denoised_unringed_sdc_b1 = ardfMRI_denoised_unringed_sdc[..., np.arange(1, ardfMRI_denoised_unringed_sdc.shape[3], 2)]
ardfMRI_denoised_unringed_sdc_b0 = ardfMRI_denoised_unringed_sdc[..., np.arange(0, ardfMRI_denoised_unringed_sdc.shape[3], 2)]


adc = np.zeros(ardfMRI_denoised_unringed_sdc_b1.shape)
# Divide b1 by b0, where b0 is not 0
np.divide(ardfMRI_denoised_unringed_sdc_b1, ardfMRI_denoised_unringed_sdc_b0, out=adc, 
          where=ardfMRI_denoised_unringed_sdc_b0!=0)
# Take the log, where adc is not 0
np.log(adc, out=adc, where=adc!=0)
adc = adc / b_val_diff

/tmp/ipykernel_20223/2984528250.py:18: RuntimeWarning: invalid value encountered in log
  np.log(adc, out=adc, where=adc!=0)


In [50]:
img_adc = nib.Nifti1Image(adc, affine_b) 
nib.save(img_adc, f'{cur_path}/data/ADC/adc_incl_weird_voxels.nii.gz') 
os.system(f'fslmerge -tr {cur_path}/data/ADC/adc_test.nii.gz {cur_path}/data/ADC/adc_incl_weird_voxels.nii.gz {2*TR}')

0

In [58]:
# Replace NaN with 0
adc = np.nan_to_num(adc)

# Replace values below 0 and above 4 by 0  
adc[adc<0] = 0
adc[adc>4] = 0

assert np.min(adc) >= 0, f"The lower thresh of 0 did not work, min = {np.min(adc)}"
assert np.max(adc) <= 4, f"The upper thresh of 4 did not work, min = {np.max(adc)}"

img_adc = nib.Nifti1Image(adc, affine_b) 
nib.save(img_adc, f'{cur_path}/data/ADC/adc.nii.gz') 

adc_mean = np.mean(adc, axis = 3)
img_adc_mean = nib.Nifti1Image(adc_mean, affine_b) 
nib.save(img_adc_mean, f'{cur_path}/data/ADC/adc_mean.nii.gz') 